In [160]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch.nn as nn
import torch

import torch_geometric as tg
from torch_geometric.transforms import NormalizeFeatures
from ogb.nodeproppred import PygNodePropPredDataset

### Check Dataloader

In [98]:
from src.dataset import *

In [106]:
dir_path = 'data'

citseer = citeSeer_pl(data_path = dir_path)
cora_ml = cora_ml_pl(data_path = dir_path)
pubmed = pubmed_pl(data_path = dir_path)
arxiv = arxiv_pl(data_path = dir_path)
flicker = flicker_pl(data_path = dir_path)
products = products_pl(data_path = dir_path)

### Model

In [216]:
from src.dataset.base import split_train_test_val

In [156]:
def split_train_test_val(label_dataset : torch.Tensor,
                        classes_num: int, 
                        each_class_num : int = 20,
                        val_num : int = 500):
    class_list = np.arange(classes_num)
    train_idx, test_idx, val_idx = torch.zeros(len(label_dataset)), torch.ones(len(label_dataset)), torch.zeros(len(label_dataset))
    find_idx_fn = lambda x : torch.nonzero(label_dataset == x, as_tuple=True)[0][:each_class_num]
    for class_idx in class_list : 
        idx = find_idx_fn(class_idx)
        train_idx[idx] = 1
        test_idx[idx] = 0
        
    _val_idx = torch.nonzero(train_idx, as_tuple=True)[0][:val_num]
    train_idx[_val_idx] = 0
    val_idx[_val_idx] = 1
    return train_idx.bool(), test_idx.bool(), val_idx.bool()

In [200]:
val_num = 500
classes_num = 7
each_class_num = 20
label_dataset = cora_ml.dataset[0].y
class_list = np.arange(classes_num)
train_idx, test_idx, val_idx = torch.zeros(len(label_dataset)), torch.ones(len(label_dataset)), torch.zeros(len(label_dataset))
find_idx_fn = lambda x : torch.nonzero(label_dataset == x, as_tuple=True)[0][:each_class_num]

In [202]:
idx = find_idx_fn(class_idx)

In [205]:
train_idx[idx] = 1

In [209]:
sum(train_idx == 1)

tensor(20)

In [75]:
model = GAT_pl()

### Test about Weights _and_Bias